# Лабораторная работа № 3
## Задача спрятанного числа

In [1]:
import numpy
from fpylll import *

### Подготовка

Задаем размерность $n$ и простое число $p$

In [2]:
n = 512
p = next_prime(2 ^ (n - 1))
p

6703903964971298549787012499102923063739682910296196688861780721860882015036773488400937149083451713845015929093243025426876941405973284973216824503042159

Определяем количество значимых бит числа $\alpha$, которые мы хотим извлечь

In [3]:
k = ceil(sqrt(n)) + ceil(log(n, 2))
k

32

Функция для извлечения из числа $x$ $k$ старших бит

In [4]:
def msb(k, x):
    xbin = x.digits(2)
    n = len(xbin)
    z = sum([2 ^ i * xbin[i] for i in range(n - k - 1, n)])
    assert abs(x - z) < p / 2 ^ (k + 1)
    return z, abs(x - z)

Функция для определения оракула
- $\alpha$ - секретное число
- $k$ - количество бит, которые мы хотим извлечь из результата

In [5]:
def create_oracle(alpha, k):
    alpha = alpha

    def oracle():
        random_t = ZZ.random_element(1, p - 1)
        ret = msb(k, (alpha * random_t) % p)
        return random_t, ret[0], ret[1]

    return oracle

Генерируем случайное $\alpha$ и создаем оракул

In [6]:
alpha = ZZ.random_element(1, p - 1)
oracle = create_oracle(alpha, k)
alpha

2240096352339036969707979822667008270091925673126520561198095261615751838856545362929062410029057481506263623594948065342994442818262547463071283315192943

Сохраняем значение $\alpha$ в файл [`alpha.txt`](http://127.0.0.1:8888/notebooks/labs/alpha.txt)

In [13]:
with open("alpha.txt", "w") as file:
    file.write(str(alpha) + "\n")

Сохраняем результат работы оракула в файл [`hidden.txt`](http://127.0.0.1:8888/notebooks/labs/hidden.txt)

In [21]:
with open(f"hidden.txt", "w") as file:
    file.write(str(p) + "\n")
    d = ceil(2 * sqrt(n) + 2)
    t = [0] * d
    msbs = [0] * d
    lsbs = [0] * d
    for i in range(d):
        t[i], msbs[i], lsbs[i] = oracle()

### Эксплуатация

Используем $k$ старших бит, полученных из оракула

In [7]:
a = msbs
a

NameError: name 'msbs' is not defined

Задаем матрицу $B$ размерности $(d + 1) \times (d + 1)$. Дополнительный столбец используется для $t_i$

In [41]:
B = Matrix(QQ, p * identity_matrix(d + 1))
B

49 x 49 dense matrix over Rational Field (use the '.str()' method to see the entries)

Заполняем матрицу $B$ значениями $t_i$, устанавливаем последний элемент последнего столбца в $\frac{1}{p}$

In [42]:
for i in range(d):
    B[i, -1] = t[i]
B[-1, -1] = 1 / p
B

49 x 49 dense matrix over Rational Field (use the '.str()' method to see the entries)

Вычисляем $t^{-1}_i$

In [43]:
t1_inv = inverse_mod(t[0], p)
t1_inv

5805346041373877142816300408179709052786454298637057921870439905505092827233621361370463031518222196353154116568077301151239130795257932393494292854713729

Вычисляем $t'_i$

In [44]:
t_cap = [(t[i] * t1_inv) % p for i in range(1, d)]
t_cap

[6526174594195002763798495436449955360315217747710481876507383039032756523318222124942046026037064046923275589826900230377466017580869382885340756251878311,
 4014919878091632119876887275227955749323257031617128118695448044316756155144982900264595927376761496838819044327873861456320346890443529189736958201283798,
 627522613407230024073099686648993431864904616241363773222511608748957571416483382465456477936638318713397011789690768666758771069597041801139996207870145,
 348768710533880823265455977020949043430479849968446372528386974479592039019735253822841574891536213957718585608626521138082212971967705396169471653696922,
 4942440511647564563788656739060655781933946777115129688956195729647301932576524589164440581928449552142977225176805825698674114483057999820007764389611897,
 311898630108396481114504974219574930537533047947812859408163492082349192748847738382002636203123420129911906021269328565810301372853805370142831120551202,
 653961568566864719665298037713250506176836045889372323045199

Вычисляем $\alpha'_i$

In [27]:
a_cap = [(a[i] - (t[i] * t1_inv * a[0])) % p for i in range(1, d)]
a_cap

[1030405831790718672193259995106696715227150695986108105228181107243088934320879222679731078993862541315798272888363609370921303112828607610739546229981403,
 2775044119267455471526112225232714741252494043008084455822057039732575092710907013492462268341940508231056778425924508631760807846430152782480086267784040,
 3612256263479373387194902635827053501040308998102842061284354631776615748359149873683717306050930210466664195007223083204382590906447704745509950588195528,
 2988139197072335087446123210568177059969053687331536162135118315972354275221131337828365026306658283046553423736882986999164006039257357183293835848109876,
 4149271889032129534863256954579384059687200496123527070263791444669877827806674474455435113755470455560539176946706397151061809132161931187022765558567422,
 3526001467649464248777600256123932123775399438140840599131906528134258960401020286263753719938420788158614952823500423818565466489255715584817158151278685,
 355717406679868773905462507716744322158375818723934892236

Создаем вектор $u$ из $\alpha'_i$ и последнего элемента

In [28]:
u = [a_cap[i] for i in range(0, d - 1)]
u.append(ceil(p / (2 ^ k)))
u

[1030405831790718672193259995106696715227150695986108105228181107243088934320879222679731078993862541315798272888363609370921303112828607610739546229981403,
 2775044119267455471526112225232714741252494043008084455822057039732575092710907013492462268341940508231056778425924508631760807846430152782480086267784040,
 3612256263479373387194902635827053501040308998102842061284354631776615748359149873683717306050930210466664195007223083204382590906447704745509950588195528,
 2988139197072335087446123210568177059969053687331536162135118315972354275221131337828365026306658283046553423736882986999164006039257357183293835848109876,
 4149271889032129534863256954579384059687200496123527070263791444669877827806674474455435113755470455560539176946706397151061809132161931187022765558567422,
 3526001467649464248777600256123932123775399438140840599131906528134258960401020286263753719938420788158614952823500423818565466489255715584817158151278685,
 355717406679868773905462507716744322158375818723934892236

Задаем матрицу $B'$, которая используется для поиска ближайшего вектора

In [29]:
B_cap = Matrix(ZZ, p * identity_matrix(d))
for i in range(d - 1):
    B_cap[-1, i] = t_cap[i]
B_cap[-1, -1] = 1
B_cap

48 x 48 dense matrix over Integer Ring (use the '.str()' method to see the entries)

Инициализируем матрицу $A$ и выполняем для нее алгоритм редукции $LLL$

In [30]:
A = IntegerMatrix.from_matrix(B_cap)
M = GSO.Mat(A, float_type="mpfr")
L = LLL.Reduction(M, delta=0.99, eta=0.501)
L()
M.update_gso()

True

Используем алгоритм Бабаи для поиска ближайшего вектора и вычисляем его

In [31]:
coef = M.babai(u)
vec = IntegerMatrix.from_iterable(1, A.ncols, coef) * A
vec2 = [vec[0, i] for i in range(d)]
vec2

[1030405831843848005505793607219770661914617410751299125508075398214680396114483748538874394980884868797559229990882290791916791134466336754370994739468044,
 2775044119868994928818020566152861785929587429073211818802341910105030755209830101673595272226597122595605870490587304033094319013589036842806495399815151,
 3612256263681223605085901645662135090352096008820952983730045263025296008181334299420615338062237343810433419871661137123567660034740271200763994026779641,
 2988139197116442634255848712404780370480014832333247102158431305926688658848797195957914951549834135548153796186792219725738571848654168022240175590323769,
 4149271889171268353441794415795418898886816725835454312420281399938620350830346153762656226457825714750233922886785293682046509551609744937648178475624936,
 3526001468030042705496500747087750615776972793132944568810051390997280877694554588495288284553615026700652559727714566041383577781628722275420516953762493,
 355717406718222835276656820896237447410438882727596658872

Вычисляем значения $b_i$

In [32]:
b_vect = [vec2e - ue for vec2e, ue in zip(vec2, u)]
b1 = ceil(p / (2**k)) - abs(b_vect[-1])
b_vect = [b1] + b_vect[0:-1]
b_vect

[297594627627733625573444840110922582605587911457483494892752371739920580614303254357222788370316193485929845474023183502576297307124137556002480,
 53129333312533612113073946687466714765191020279894290971591461793604525859143315987022327481760957102518681420995488021637729143631448509486641,
 601539457291908340920147044677093386065127362980284870372455662498923088181133003884656614364549092064662795401333511167158884060326409132031111,
 201850217890999009835081589311787010718110922445690631248680259822184425736898032011307133343769224864438053919185069128292566455254043438584113,
 44107546809725501836603310510961145001710940023312989954334383627665858129549925243175852501600372449909232726574565809396810838946339742213893,
 139138818578537461216034839199616229711927242156489955268742523023671679307221112702355259189694745940078896530984700419447813750625412917057514,
 38057845671890049096381849200157335499210396967814486286302191729353430223153456461519423854203760690421414222281811129

Вычисляем значение $\alpha$ и проверяем совпадение

In [33]:
alpha_answer = (((a[i] + b_vect[i]) % p) * inverse_mod(t[i], p)) % p
alpha_answer == alpha

True